In [ ]:
import time

from djitellopy import tello
import KeyPressModule as Kp
import numpy as np
from time import sleep
import cv2
import math

#######Parameters########
fSpeed = 117/10 # ForwardnSpped in cm/s (15 cm/s)
aSpeed = 360/10 # Angular Speed Degrees/s (50d/s)
interval = 0.25

dInterval = fSpeed*interval
aInterval = aSpeed*interval

#####################################

x, y = 500, 500
a = 0
yaw = 0
Kp.init()
me = tello.Tello()
me.connect()
print(me.get_battery())

points = [(0, 0), (0, 0)]


def getKeyboardInput():
    lr, fb, ud, yv = 0, 0, 0, 0

    speed = 15
    aspeed = 50
    global x, y, yaw, a
    d = 0

    if Kp.getKey("LEFT"):
        lr = -speed
        d = dInterval
        a = -180
    elif Kp.getKey("RIGHT"):
        lr = speed
        d = -dInterval
        a = 180

    if Kp.getKey("UP"):
        fb = speed
        d = dInterval
        a = 270

    elif Kp.getKey("DOWN"):
        fb = -speed
        d = -dInterval
        a = -90

    if Kp.getKey("w"):
        ud = speed
    elif Kp.getKey("s"):
        ud = -speed

    if Kp.getKey("a"):
        yv = -aspeed
        yaw -= aInterval

    elif Kp.getKey("d"):
        yv = aspeed
        yaw += aInterval

    if Kp.getKey("q"): me.land()
    if Kp.getKey("e"): me.takeoff()
    sleep(interval)
    a += yaw
    x += int(d* math.cos(math.radians(a)))
    y += int(d* math.sin(math.radians(a)))


    return [lr, fb, ud, yv, x, y]
def drawPoints(img, points):
    for point in points:
        cv2.circle(img, point, 5, (0, 0, 255)), cv2.FILLED
        cv2.circle(img, points[-1], 5, (0, 255, 0)), cv2.FILLED

    cv2.putText(img, f'({(points[-1][0] - 500 )/ 100},{(points[-1][1] - 500) / 100})m',(points[-1][0] + 10, points[-10][1] + 30), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 255), 1)

while True:
    vals = getKeyboardInput()
    me.send_rc_control(vals[0], vals[1], vals[2], vals[3])


    img = np.zeros((1000, 1000, 3), np.vint8)
    if(points[-1][0] != vals[4] or points[-1][1] != vals[5]):
         points.append((vals[4], vals[5]))
    drawPoints(img, points)
    cv2.imshow("Output", img)
    cv2.waitKey(1)

In [ ]:
import cv2
import numpy as np
import utlis

###############################

drone = False
path = 'Image.jpg'
cap = cv2.VideoCapture(0)
cap.set(10, 160)
cap.set(3, 1920)
cap.set(4, 1080)
scale = 3
wP = 210*scale
hP = 297*scale

while True:
    if drone:sucess,img = cap.read()
    else: img = cv2.imread(path)
    img, conts = utlis.getContours(img,showCanny=True,minArea=50000,filter=4)
    if len(conts) != 0:
        biggest = conts[0][2]
        #print(biggest)
        imgWrap = utlis.warpImg(img,biggest,wP,hP)
        imgContours2, conts2 = utlis.getContours(imgWrap,
                                                  minArea=2000,filter=4,
                                                  cThr=[50,50],draw = False)

        if len(conts) != 0:
            for obj in conts2:
                cv2.polylines(imgContours2,[obj[2]],True(0,255,0),2)
                nPoints = utlis.reorder(obj[2])
                nW = round((utlis.findDis(nPoints[0][0]//scale,nPoints[1][0]//scale)/10),1)
                nH = round((utlis.findDis(nPoints[0][0]//scale,nPoints[2][0]//scale)/10),1)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0]),nPoints[0][0][1]), (nPoints[1][0][0], nPoints(255, 0 ,255),3,8,0, 0.05)
                cv2.arrowedLine(imgContours2, (nPoints[0][0][0], nPoints[0][0][1]), (nPoints[2][0][0], nPoints(255, 0 ,255),3,8, 0, 0.05))
                x, y, w, h = obj[3]
                cv2.putText(imgContours2, '{}cm'.format(nW), (x + 30, y - 10),cv2.FONT_HERSHEY_COMPLEX_SMALL(255, 0, 255),2)
                cv2.putText(imgContours2, '{}cm'.format(nH), (x - 70, y + h//2),cv2.FONT_HERSHEY_COMPLEX_SMALL(255, 0, 255),2)






        cv2.imshow('A4',imgContours2)

    img = cv2.resize(img, (0,0), None, 0.5, 0.5)

    cv2.imshow('Original',img)
    cv2.waitKey(1)

In [ ]:
import cv2
import numpy as np

def getContours(img,cThr=[100,100],showCanny=False,minArea=1000,filter=0,draw = False):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),1)
    imgCanny = cv2.Canny(imgBlur,cThr[0],cThr[1])
    kernel = np.ones((5, 5))
    imgDial = cv2.dilate(imgCanny,kernel,iterations=3)
    imgThre = cv2.erode(imgDial,kernel,iterations=2)
    if showCanny:cv2.imshow('Canny',imgThre)

    contours,hiearchy = cv2.findContours(imgThre,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    finalCountours = []

    for i in contours:
        area = cv2.contourArea(i)
        if area > minArea:
            peri = cv2.arcLength(i,True)
            approx = cv2.approxPolyDP(i,0.02*peri,True)
            bbox = cv2.boundingRect(approx)
            if filter > 0:
                if len(approx) == filter:
                    finalCountours.append(len(approx),area,approx,bbox,i)
                else:
                    finalCountours.append(len(approx),area,approx,bbox,i)
            finalCountours = sorted(finalCountours,key = lambda x:x[1], reverse = True)
            if draw:
                for con in finalCountours:
                    cv2.drawContours(img,con[4],-1,(0, 0, 255),3)
            return img, finalCountours
        def reorder(myPoints):
            print(mypoints.shape)
            myPointsNew = np.zeros_like(myPoints)
            myPoints = myPoints.reshape((4,2))
            add = myPoints.sum(1)
            myPointsNew[0] = myPoints[np.argmin(add)]
            myPoints[3] = myPoints[np.argmax(add)]
            diff = np.diff(myPoints,axis=1)
            myPointsNew[1] = myPoints[np.argmin(diff)]
            myPointsNew[2] = myPoints[np.argmax(diff)]
            return myPointsNew



        def warpImg (img,points,w,h,pad=20):
            #print(points)
            points = reorder(points)

            pts1 = np.float32(points)
            pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
            matrix = cv2.getPerspectiveTransform(pts1,pts2)
            imgWarp = cv2.warpPerspective(img,matrix,(w,h))
            imgWarp = imgWarp[pad:imgWarp.shape[0]-pad,pad:imgWrap.shape[1]-pad]

            return imgWarp
    def findDis(pts1,pts2):
        return ((pts2[0]-pts1[0])*2 + (pts2[1]-pts1[1]2)*0.5)